In [22]:
import pandas as pd
import tensorflow as tf

In [5]:
ratings = pd.read_csv('C://Projects//Movie Recommender System//Datasets//ratings.csv')

In [14]:
movies = pd.read_csv('C://Projects//Movie Recommender System//Datasets//movies_metadata.csv')

C:\Users\Nihar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [8]:
ratings.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

In [11]:
train_X,test_X = train_test_split(ratings, test_size=0.2)

In [12]:
train_X

,userId,movieId,rating,timestamp
7159834,73875,1593,3.0,958210061
12421332,128838,543,1.5,1144673944
7493405,77327,45672,3.0,1463697584
8662342,89263,6695,2.0,1229807065
4405632,45521,3030,5.0,1258901884
...,...,...,...,...
16212796,168540,5266,3.5,1416612440
24917229,259065,1704,4.0,1164563093
22139524,229884,2353,4.0,963175959
11484916,119153,7361,5.0,1488438090


In [13]:
test_X

,userId,movieId,rating,timestamp
4298828,44587,1573,4.0,954483429
15578790,162180,858,5.0,1387157639
11019465,113962,6947,3.5,1311061147
6604331,68158,5991,3.0,1453916650
12713352,131973,4008,4.0,975743979
...,...,...,...,...
21042488,218687,3706,4.0,1214668465
1904835,19777,5266,3.0,1447940672
11075163,114579,454,3.0,846865616
18528426,192323,2289,1.5,1180568197


In [34]:
total_movies = ratings.movieId.nunique()
total_users = ratings.userId.nunique()
print(total_movies)
print(total_users)

45115
270896


In [ ]:
45436
270896

In [18]:
embedding_dimension = 32

In [35]:
#Movies Network
input_movies = tf.keras.layers.Input(shape=[1])
embed_movies = tf.keras.layers.Embedding(total_movies + 1, embedding_dimension)(input_movies)
out_movies = tf.keras.layers.Flatten()(embed_movies)
#Users Network
input_users = tf.keras.layers.Input(shape=[1])
embed_users = tf.keras.layers.Embedding(total_users + 1, embedding_dimension)(input_users)
out_users = tf.keras.layers.Flatten()(embed_users)

conc_layer = tf.keras.layers.Concatenate()([out_movies,out_users])
x = tf.keras.layers.Dense(128, activation='relu')(conc_layer)
x_out = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.Model([input_movies,input_users], x_out)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mean_squared_error')

In [36]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 1, 32)        1443712     input_13[0][0]                   
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 1, 32)        8668704     input_14[0][0]                   
____________________________________________________________________________________________

In [37]:
hist = model.fit([train_X.movieId,train_X.userId], train_X.rating,
                 batch_size=64,epochs=5,
                 verbose=2,
                 validation_data= ([test_X.movieId,test_X.userId], test_X.rating))

Epoch 1/5


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  indices[2,0] = 55577 is not in [0, 45116)
	 [[node model_3/embedding_11/embedding_lookup (defined at <ipython-input-37-b0659f84cb3f>:4) ]]
	 [[model_3/embedding_11/embedding_lookup/_24]]
  (1) Invalid argument:  indices[2,0] = 55577 is not in [0, 45116)
	 [[node model_3/embedding_11/embedding_lookup (defined at <ipython-input-37-b0659f84cb3f>:4) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_2095]

Errors may have originated from an input operation.
Input Source operations connected to node model_3/embedding_11/embedding_lookup:
 model_3/embedding_11/embedding_lookup/1812 (defined at C:\Users\Nihar\Anaconda3\lib\contextlib.py:112)

Input Source operations connected to node model_3/embedding_11/embedding_lookup:
 model_3/embedding_11/embedding_lookup/1812 (defined at C:\Users\Nihar\Anaconda3\lib\contextlib.py:112)

Function call stack:
train_function -> train_function
